<a href="https://colab.research.google.com/github/KieranPereira/SkinBurnClassification/blob/main/Studentship_Code_(Yolov5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

**Aim:**

With medical clinics and hospitals being under constant strain, the need for on-demand medical advice has never been greater. This has led many people to seek health advice on the internet, something that was usually dismissed due to unreliable or conflicting answers. Today, apps like Ada, a symptom checker, are only increasing in demand, with 5 million + downloads on the android store to date. Inevitably, these solutions for quick health advice will only increase due it its anonymity and data from these sources being verified by qualified doctors.

Last term, I had several disappointing experiences with NHS response times. One of my friends received 2nd-degree burns on her leg after spilling boiling tea, after calling for an ambulance, she was told that there would be a 5 hour waiting period before an ambulance could be dispatched. She was unfortunately on her own and had to find out how to appropriately treat the burn before the doctors could come. I understand that the NHS is underfunded but was shocked by the slow response to such a serious injury and decided to do something about this. I am interested in using image recognition software to give a basic diagnosis for burns, rashes and wounds.

**NOTE:**

**The following notebook is work created during a summer research project sponsored by UCL**


Prerequisites: Must have kaggle API token downloaded (see below for instructions)

# **Installing Dependencies & Imports**

Installing Dependencies

In [1]:
import torch
import os
import shutil
import zipfile
import time
import yaml
%pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Cloning yolov5 repo and installing additional dependencies

In [2]:
#Cloning Yolov5 Repository
if os.path.exists("yolov5")==False:

    print("preparing to clone yolov5 repository")
    time.sleep(1)

    !git clone https://github.com/ultralytics/yolov5
    os.chdir("yolov5")
    #Installing Dependencies

    print("preparing to install dependencies")
    time.sleep(1)

    %pip install -r requirements.txt
    os.chdir("..")


preparing to clone yolov5 repository
Cloning into 'yolov5'...
remote: Enumerating objects: 13687, done.
remote: Counting objects: 100% (489/489), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 13687 (delta 354), reused 435 (delta 310), pack-reused 13198
Receiving objects: 100% (13687/13687), 13.29 MiB | 30.17 MiB/s, done.
Resolving deltas: 100% (9413/9413), done.
preparing to install dependencies
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 41.0 MB/s 


# **Dataset Paths and formatting**

Creating Dataset Paths for Training, Validation and Testing. Creating Subdirectories for images and labels in each folder.

In [ ]:
#Creating dataset paths
if os.path.exists("BurnsDataset")==False:
    
    print("Preparing to create dataset paths")
    
  
    os.makedirs("BurnsDataset")
    os.chdir("BurnsDataset")
    os.makedirs("test/images")
    os.makedirs("test/labels")
    os.makedirs("train/images")
    os.makedirs("train/labels")
    os.makedirs("val/images")
    os.makedirs("val/labels")
    os.chdir("..")

Preparing to create dataset paths


In [ ]:
#Navigating to home directory
while os.getcwd()!="/content":
  os.chdir("..")
print(os.getcwd())

/content


Downloading Dataset from kaggle.com. Requires API token to be downloaded and uploaded to drive. Instructions for downloading API token can be found here: https://www.kaggle.com/docs/api

In [ ]:

if os.path.exists("..\.kaggle\kaggle.json")==False:
    
    print("preparing to download dataset")
    time.sleep(2)
    !mkdir ~/.kaggle
    from google.colab import files
    files.upload()
    shutil.move("kaggle.json","../root/.kaggle/kaggle.json")
    !chmod 600 ~/.kaggle/kaggle.json
    !kaggle datasets download -d shubhambaid/skin-burn-dataset
    with zipfile.ZipFile("skin-burn-dataset.zip","r") as zip_ref:
        zip_ref.extractall("BurnsDataset")
    os.remove("skin-burn-dataset.zip")

preparing to download dataset


Saving kaggle.json to kaggle.json
  0% 0.00/16.6M [00:00<?, ?B/s]
100% 16.6M/16.6M [00:00<00:00, 195MB/s]


In [ ]:
os.chdir("/content")

Finding number of valid/usable images in the dataset.

In [ ]:
#Dataset formatting
temp_len= int(len(os.listdir("BurnsDataset")))
print(temp_len)
#Dataset formatting
os.chdir("BurnsDataset")
print("preparing to format data")
#Removing unlabelled images from dataset
#Counting number of images in new dataset
number_img = 0
text_img =0
for i in range (0,int(temp_len)+1):
  string1= str(i)
  if os.path.exists("img"+string1+".jpg")==True:
      number_img=number_img+1

print("There are {} images in your dataset".format(str(number_img)))

2671
preparing to format data
There are 1225 images in your dataset


Defining Ratios for Test:Validation:Train data. 

*Note: For improved training results, increase the ratio for 'Train' data.*

In [ ]:
    print("preparing to format data")
    test_ratio=float(input("Please enter the ratio for your test data (note test, val, train data must sum to 10)"))
    val_ratio=float(input("Please enter the ratio for your val data (note test, val, train data must sum to 10)"))
    train_ratio=float(input("Please enter the ratio for your train data (note test, val, train data must sum to 10)"))
    while test_ratio+val_ratio+train_ratio !=10:
        print(" test, val, train data must sum to 10")
        test_ratio=float(input("Please enter the ratio for your test data (note test, val, train data must sum to 10)"))
        val_ratio=float(input("Please enter the ratio for your val data (note test, val, train data must sum to 10)"))
        train_ratio=float(input("Please enter the ratio for your train data (note test, val, train data must sum to 10)"))
    print("splitting into " +str(test_ratio)+":"+str(val_ratio)+":" +str(train_ratio)+" for test:val:train")
    time.sleep(2)
    test_num= number_img//(float(1/(test_ratio/10)))
    val_num= number_img//(float(1/(val_ratio/10)))
    train_num= number_img//(float(1/(train_ratio/10)))

preparing to format data
Please enter the ratio for your test data (note test, val, train data must sum to 10)0.5
Please enter the ratio for your val data (note test, val, train data must sum to 10)1.5
Please enter the ratio for your train data (note test, val, train data must sum to 10)8
splitting into 0.5:1.5:8.0 for test:val:train


Moving images and associated labels to test,train and validation folders

In [ ]:
    while os.getcwd()!="/content":
      os.chdir("..")
    print(os.getcwd())
# create a list of files in burns dataset and appending it to a .txt file. This allows  
    allFiles=[]
    for entry in os.listdir("BurnsDataset"):
        if ".jpg" in entry:
            allFiles.append(entry)
        if ".jpeg" in entry:
            allFiles.append(entry)
    with open("allfiles.txt","w") as w:
        w.write(str(allFiles))
    w.close
    te_num=0
    va_num=0
    tr_num=0
    os.chdir("BurnsDataset")
    print("appending files to test folder: \n {}".format(os.getcwd()))
    time.sleep(2)
    for i in range(0,len(allFiles)):
        if i in range(0,int(test_num)):
            te_num=te_num+1
            entry_file= allFiles[i]
            if ".jpg" in entry_file:
                shutil.move(entry_file, "test")
                shutil.move(str(entry_file).replace(".jpg",".txt"), "test")
            elif ".jpeg" in entry_file:
                shutil.move(entry_file, "test")
                shutil.move(str(entry_file).replace(".jpeg",".txt"),"test")
        os.chdir("test")
        for filename in os.listdir():
            if ".jpg" in filename:
                shutil.move(filename,"images")
            elif ".txt" in filename:
                shutil.move(filename,"labels")
            elif ".jpeg" in filename:
                shutil.move(filename,"images")
        os.chdir("..")
    print("te_num={}".format(te_num))
    print("appending files to validation folder: \n {}".format(os.getcwd()))
    time.sleep(2)
    for i in range(0,len(allFiles)):
        if i in range(int(test_num),int(val_num*2)):
            va_num=va_num+1
            entry_file= allFiles[i]
            if ".jpg" in entry_file:
                shutil.move(entry_file, "val")
                shutil.move(str(entry_file).replace(".jpg",".txt"), "val")
            elif ".jpeg" in entry_file:
                shutil.move(entry_file, "val")
                shutil.move(str(entry_file).replace(".jpeg",".txt"), "val")
        os.chdir("val")
        for filename in os.listdir():
            if ".jpg" in filename:
                shutil.move(filename,"images")
            elif ".txt" in filename:
                shutil.move(filename,"labels")
            elif ".jpeg" in filename:
                shutil.move(filename,"images")
        os.chdir("..")
    print("va_num={}".format(va_num))
    print("appending files to train folder: \n {}".format(os.getcwd()))
    time.sleep(2)
    for i in range(0,len(allFiles)):
        if i in range(int(val_num*2),len(allFiles)):
            tr_num=tr_num+1
            entry_file= allFiles[i]
            if ".jpg" in entry_file:
                shutil.move(entry_file, "train")
                shutil.move(str(entry_file).replace(".jpg",".txt"), "train")
            elif ".jpeg" in entry_file:
                shutil.move(entry_file, "train")
                shutil.move(str(entry_file).replace(".jpeg",".txt"), "train")
        os.chdir("train")
        for filename in os.listdir():
            if ".jpg" in filename:
                shutil.move(filename,"images")
            elif ".txt" in filename:
                shutil.move(filename,"labels")
            elif ".jpeg" in filename:
                shutil.move(filename,"images")
        os.chdir("..")
    for file in os.listdir():
        if ".txt" in file:
            os.remove(file)
    print("Dataset Successfully Sorted")

/content
appending files to test folder: 
 /content/BurnsDataset
te_num=61
appending files to validation folder: 
 /content/BurnsDataset
va_num=305
appending files to train folder: 
 /content/BurnsDataset
Dataset Successfully Sorted


#**Defining Custom Architecture and defining data pathways**

Creating Custom Yaml file for data paths

In [ ]:

while os.getcwd()!="/content":
  os.chdir("..")
print(os.getcwd())
if os.path.exists("yolov5/data/custom.yaml") == False:
    print("Creating custom yaml file:")
    time.sleep(2)
    with open('custom.yaml', 'w+') as file:
        file.write(
"""
path: ../BurnsDataset
train: train/images
test: test/images
val: val/images
nc: 3
names: ["1st Degree","2nd Degree","3rd Degree"]
"""
        )
        file.close()
    shutil.move("custom.yaml","yolov5/data/custom.yaml")

/content
Creating custom yaml file:


Defining Custom Architecture in a yaml file

In [ ]:
num_classes = 3
##write custom model .yaml
with open('custom_yolov5s.yaml', 'w') as f:
  # parameters
  f.write('nc: ' + str(num_classes) + '\n')
  #f.write('nc: ' + str(len(class_labels)) + '\n')
  f.write('depth_multiple: 0.33'  + '\n') # model depth multiple
  f.write('width_multiple: 0.50'  + '\n')  # layer channel multiple
  f.write('\n')
  f.write('anchors:' + '\n')
  f.write('  - [10,13, 16,30, 33,23] ' + '\n')
  f.write('  - [30,61, 62,45, 59,119]' + '\n')
  f.write('  - [116,90, 156,198, 373,326] ' + '\n')
  f.write('\n')

  f.write('backbone:' + '\n')
  f.write('  [[-1, 1, Focus, [64, 3]],' + '\n') #0-P1/2
  f.write('   [-1, 1, Conv, [128, 3, 2]],' + '\n')# 1-P2/4
  f.write('   [-1, 3, BottleneckCSP, [128]],' + '\n')
  f.write('   [-1, 1, Conv, [256, 3, 2]],' + '\n')# 3-P3/8
  f.write('   [-1, 9, BottleneckCSP, [128]],' + '\n')
  f.write('   [-1, 1, Conv, [512, 3, 2]], ' + '\n')# 5-P4/16
  f.write('   [-1, 9, BottleneckCSP, [256]],' + '\n')
  f.write('   [-1, 1, Conv, [1024, 3, 2]],' + '\n')# 7-P5/32
  f.write('   [-1, 1, SPP, [1024, [5, 9, 13]]],' + '\n')
  f.write('   [-1, 3, BottleneckCSP, [1024,False]],' + '\n') # 9
  f.write('  ]' + '\n')
  f.write('\n')

  f.write('head:'  + '\n')
  f.write('  [[-1, 1, Conv, [512, 1, 1]],'  + '\n')
  f.write('   [-1, 1, nn.Upsample, [None, 2,\'nearest\']],' + '\n')
  f.write('   [[-1, 6], 1, Concat, [1]],' + '\n')#Cat Backbone P4
  f.write('   [-1, 3, BottleneckCSP, [512, False]],' + '\n')# 13
  f.write('\n' )
  f.write('   [-1, 1, Conv, [256, 1, 1]],'  + '\n')
  f.write('   [-1, 1, nn.Upsample, [None, 2, \'nearest\']],' + '\n')
  f.write('   [[-1, 4], 1, Concat, [1]],' + '\n')# cat backbone P3
  f.write('   [-1, 3, BottleneckCSP, [256, False]],' + '\n') # 17 (P3/8-small)
  f.write('\n' )
  f.write('   [-1, 1, Conv, [256, 3, 2]],' + '\n')
  f.write('   [[-1, 14], 1, Concat, [1]],' + '\n')# cat head P4
  f.write('   [-1, 3, BottleneckCSP, [512, False]],' + '\n')# 23 (P5/32-large)
  f.write('\n' )
  f.write('   [-1, 1, Conv, [512, 3, 2]],' + '\n')
  f.write('   [[-1, 10], 1, Concat, [1]],' + '\n')# cat head P5
  f.write('   [-1, 3, BottleneckCSP, [1024, False]],' + '\n')# 23 (P5/32-large)
  f.write('\n' )
  f.write('   [[17, 20, 23], 1, Detect, [nc, anchors]],' + '\n')# Detect(P3, P4, P5)
  f.write('  ]' + '\n')

print('custom model config written')
shutil.move("custom_yolov5s.yaml", "/content/yolov5/models/custom_yolov5s.yaml")

custom model config written!


'/content/yolov5/models/custom_yolov5s.yaml'

In [ ]:
print(os.getcwd())

/content


In [ ]:
#Changing Directory to facilitate training
while os.getcwd()!="/content":
  os.chdir("..")
os.chdir("yolov5")
print(os.getcwd())

/content/yolov5


# **Training Model**

In [ ]:
#Training custom model
!python train.py --data custom.yaml --cfg custom_yolov5s.yaml --weights '' --batch-size 32 --epochs 600 --img 640 --cache 

train: weights=, cfg=custom_yolov5s.yaml, data=custom.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-155-g489920a Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv

In [ ]:
#Hyperparameter optimisation of custom model (must be done after initial training has completed)
!python train.py --data custom.yaml --resume /content/yolov5/runs/train/exp5/weights/last.pt --batch-size 32 --epochs 20 --img 640 --cache --hyp hyp.scratch-high.yaml --evolve

train: weights=yolov5s.pt, cfg=, data=custom.yaml, hyp=hyp.scratch-high.yaml, epochs=1, batch_size=16, imgsz=640, rect=False, resume=/content/yolov5/runs/train/exp5/weights/last.pt, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=300, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-155-g489920a Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.3, cls_pw=1.0, obj=0.7, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=

In [ ]:
#Training using pre-trained weights from YOLOV5 repository
# !python train.py --data custom.yaml --weights yolov5s.pt --batch-size 32 --epochs 100 --img 640 --cache 

# **Testing model**

Detecting through webcam. Note: unavailable on colab, must download weights and yolov5 repository locally.

In [ ]:
!python detect.py --source 0 --weights runs/train/exp/weights/best.pt

detect: weights=['runs/train/exp/weights/best.pt'], source=0, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-155-g489920a Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 5893864 parameters, 0 gradients
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show() image displays
cv2.imshow() is disabled in Docker environments
Traceback (most recent call last):
  File "detect.py", line 255, in <module>
    main(opt)
  File "detect.py", line 250, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/grad_mode.py", line 

Detecting a custom image: Image must be downloaded first

In [ ]:
if os.path.exists("/content/yolov5/Test_images")==False:
  os.mkdir("Test_images")
  os.chdir("Test_images")
  from google.colab import files
  files.upload()
  for file in os.listdir():
    os.rename(file,"test_img.jpg")
os.chdir("/content/yolov5")
!python detect.py --source /content/yolov5/Test_images/test_img.jpg --weights runs/train/exp/weights/best.pt
shutil.rmtree("/content/yolov5/Test_images")

Saving burn_test.jpg to burn_test.jpg
detect: weights=['runs/train/exp/weights/best.pt'], source=/content/yolov5/Test_images/test_img.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-155-g489920a Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 5893864 parameters, 0 gradients
image 1/1 /content/yolov5/Test_images/test_img.jpg: 320x640 (no detections), 12.8ms
Speed: 0.4ms pre-process, 12.8ms inference, 0.3ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp
